In [ ]:
!pip install -q keras-nlp --upgrade


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import keras_nlp
import keras
import tensorflow as tf
import os
import gc

keras.mixed_precision.set_global_policy("mixed_float16")


In [5]:
DATA_DIR = '/kaggle/input/contradictory-my-dear-watson/'
for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df_train = pd.read_csv(DATA_DIR +"train.csv")
df_train.head()


In [ ]:
def split_labels(x, y):
    return (x[0], x[1]), y


- Use this block if the preprocessor used for the model is changed from bert base multi

In [ ]:
batch_size = 36
buffer_size = batch_size * 10

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_train[["hypothesis", "premise"]].values,
            df_train["label"].values
        )
    )
)

print(f"Size of the train dataset: {len(training_dataset)}")

train_preprocessed = training_dataset.shuffle(buffer_size=buffer_size).map(split_labels, tf.data.AUTOTUNE).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)


In [ ]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en", sequence_length=195)

bert_train_set = (train_preprocessed.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE))

classifier = keras_nlp.models.BertClassifier.from_preset("bert_base_en", preprocessor=None, num_classes=3)
classifier.compile(optimizer=keras.optimizers.Adam(4e-05), metrics=['accuracy'])
history = classifier.fit(bert_train_set, epochs=5)


### Precitions on test data for submission

In [ ]:
df_test = pd.read_csv(DATA_DIR +"test.csv")
print(df_test.head())
df_test["label"] = [0] * len(df_test)


In [ ]:
def split_labels(x, y):
    return (x[0], x[1]), y


testing_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_test[["hypothesis", "premise"]].values,
            df_test["label"].values
        )
    )
)

print(len(testing_dataset))
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en", sequence_length=195)

test_preprocessed = testing_dataset.map(split_labels, tf.data.AUTOTUNE).batch(1, drop_remainder=False).cache().prefetch(tf.data.AUTOTUNE)
bert_test_set = (test_preprocessed.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE))


In [ ]:
predictions = classifier.predict(bert_test_set)

In [ ]:
predicted_classes = tf.argmax(predictions, axis=1)
predicted_classes_np = predicted_classes.numpy()
print(predicted_classes_np)

In [ ]:
submission = df_test.id.copy().to_frame()
submission["prediction"] = predicted_classes_np
submission.to_csv("submission.csv", index=False)

submission